In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.utils import np_utils
from keras import metrics

In [48]:
data = pd.read_csv('/content/drive/My Drive/2022-1/기계학습/master_3_1.csv', index_col=0, encoding='cp949')

In [7]:
print(data.isna().sum())
print(data.shape)
print(list(data.columns))

subject_id         0
stay_id            0
gender             0
in_year            0
age                0
                  ..
ed_dbp_last        0
ed_pain_last       0
n_med              0
n_medrecon         0
critical_result    0
Length: 79, dtype: int64
(448972, 79)
['subject_id', 'stay_id', 'gender', 'in_year', 'age', 'outcome_inhospital_mortality', 'ed_death', 'before_ed_mortality', 'outcome_hospitalization', 'n_ed_30d', 'n_ed_90d', 'n_ed_365d', 'outcome_ed_revisit_3d', 'n_hosp_30d', 'n_hosp_90d', 'n_hosp_365d', 'n_icu_30d', 'n_icu_90d', 'n_icu_365d', 'ed_los_hours', 'triage_temperature', 'triage_heartrate', 'triage_resprate', 'triage_o2sat', 'triage_sbp', 'triage_dbp', 'triage_pain', 'triage_acuity', 'chiefcom_chest_pain', 'chiefcom_abdominal_pain', 'chiefcom_headache', 'chiefcom_shortness_of_breath', 'chiefcom_back_pain', 'chiefcom_cough', 'chiefcom_nausea_vomiting', 'chiefcom_fever_chills', 'chiefcom_syncope', 'chiefcom_dizziness', 'eci_CHF', 'eci_Arrhythmia', 'eci_Valvular', 'e

In [49]:
data['death_result'] = data['ed_death'] 

In [50]:
data=data.drop('ed_death',axis=1)
data

,subject_id,stay_id,gender,in_year,age,outcome_inhospital_mortality,before_ed_mortality,outcome_hospitalization,n_ed_30d,n_ed_90d,...,ed_heartrate_last,ed_resprate_last,ed_o2sat_last,ed_sbp_last,ed_dbp_last,ed_pain_last,n_med,n_medrecon,critical_result,death_result
index,,,,,,,,,,,,,,,,,,,,,
335314,10000032,33258284,0,2180,52,False,False,True,0,0,...,79.0,16.0,98.0,107.0,60.0,0.0,0.0,9.0,False,False
335315,10000032,38112554,0,2180,52,False,False,True,0,1,...,86.0,17.0,93.0,96.0,57.0,5.0,3.0,12.0,False,False
335316,10000032,32952584,0,2180,52,False,False,True,1,2,...,85.0,18.0,98.0,81.0,38.0,0.0,2.0,14.0,True,False
335317,10000032,39399961,0,2180,52,False,False,True,2,3,...,96.0,18.0,97.0,86.0,45.0,0.0,8.0,14.0,True,False
335318,10000032,35968195,0,2180,52,False,False,True,2,3,...,91.0,18.0,99.0,98.0,60.0,0.0,6.0,7.0,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167406,19999784,34149746,1,2119,57,False,False,True,0,2,...,72.0,16.0,100.0,131.0,92.0,0.0,1.0,8.0,False,False
167407,19999828,30712109,0,2147,46,False,False,True,0,0,...,78.0,17.0,99.0,108.0,87.0,9.0,11.0,8.0,False,False
167408,19999828,32917002,0,2149,48,False,False,True,0,0,...,109.0,15.0,96.0,111.0,78.0,4.0,8.0,11.0,True,False


In [59]:
# x y 로 데이터 분리 후 dmatrix로 변환 (y1=치명적인지 아닌지 y2= 부상정도 )
data_x = data.iloc[:,1:-2]
data_y = pd.concat([data['critical_result'], data['death_result']], axis=1)
data_x.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 448972 entries, 335314 to 335313
Data columns (total 76 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   stay_id                       448972 non-null  int64  
 1   gender                        448972 non-null  int64  
 2   in_year                       448972 non-null  int64  
 3   age                           448972 non-null  int64  
 4   outcome_inhospital_mortality  448972 non-null  bool   
 5   before_ed_mortality           448972 non-null  bool   
 6   outcome_hospitalization       448972 non-null  bool   
 7   n_ed_30d                      448972 non-null  int64  
 8   n_ed_90d                      448972 non-null  int64  
 9   n_ed_365d                     448972 non-null  int64  
 10  outcome_ed_revisit_3d         448972 non-null  bool   
 11  n_hosp_30d                    448972 non-null  int64  
 12  n_hosp_90d                    448972 no

In [61]:
from sklearn.preprocessing import LabelEncoder

In [54]:


# 레이블 데이터
input_labels = ['False','True']
encoder = LabelEncoder()

# 레이블 학습
encoder.fit(input_labels)
for i, label in enumerate(encoder.classes_):
    print(i, '->', label)

# 문자를 레이블로 변환
test_labels = data['critical_result']
data['critical_result'] =  encoder.fit_transform(data['critical_result'].values)
data['death_result'] =  encoder.fit_transform(data['death_result'].values)
data['critical_result']

0 -> False
1 -> True


index
335314    0
335315    0
335316    1
335317    1
335318    0
         ..
167406    0
167407    0
167408    1
167409    0
335313    1
Name: critical_result, Length: 448972, dtype: int64

In [62]:

data_y.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 448972 entries, 335314 to 335313
Data columns (total 2 columns):
 #   Column           Non-Null Count   Dtype
---  ------           --------------   -----
 0   critical_result  448972 non-null  int64
 1   death_result     448972 non-null  int64
dtypes: int64(2)
memory usage: 10.3 MB


In [65]:
data_x = data_x.astype(np.float32)
data_x.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 448972 entries, 335314 to 335313
Data columns (total 76 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   stay_id                       448972 non-null  float32
 1   gender                        448972 non-null  float32
 2   in_year                       448972 non-null  float32
 3   age                           448972 non-null  float32
 4   outcome_inhospital_mortality  448972 non-null  float32
 5   before_ed_mortality           448972 non-null  float32
 6   outcome_hospitalization       448972 non-null  float32
 7   n_ed_30d                      448972 non-null  float32
 8   n_ed_90d                      448972 non-null  float32
 9   n_ed_365d                     448972 non-null  float32
 10  outcome_ed_revisit_3d         448972 non-null  float32
 11  n_hosp_30d                    448972 non-null  float32
 12  n_hosp_90d                    448972 no

In [12]:
print(data_y.isna().sum())

critical_result    0
acuity_result      0
dtype: int64


In [58]:
data_y.head()

,critical_result,death_result
index,,
335314,False,False
335315,False,False
335316,True,False
335317,True,False
335318,False,False


In [63]:
epoch = [32, 64, 128]
batch_sizes = [8, 16, 32]

In [67]:
for i in epoch:
    for j in batch_sizes:
        model_file_name = "mt_model_"+str(i)+"_"+str(j)
        print("Model: "+model_file_name)
        print("Epoch: "+str(i)+", Batch size: "+str(j))

        model = Sequential()
        model.add(Dense(128, input_shape=(76,), activation='relu', kernel_initializer='normal'))
        model.add(Dropout(rate=0.1))
        model.add(Dense(128, input_shape=(76,), activation='relu', kernel_initializer='normal'))
        model.add(Dense(128, activation='relu', kernel_initializer='normal'))
        model.add(Dense(data_y.shape[1], activation='sigmoid', kernel_initializer='normal'))

        model.compile(loss='binary_crossentropy', optimizer='adam',  metrics=['accuracy'])

        model.fit(data_x, data_y, epochs=i, batch_size=j, validation_split=0.2, verbose=2)

        y = model.predict(data_x)

        model.save(model_file_name)
        print()

Model: mt_model_32_8
Epoch: 32, Batch size: 8
Epoch 1/32
44898/44898 - 123s - loss: 13.8726 - accuracy: 0.9998 - val_loss: 0.1219 - val_accuracy: 1.0000 - 123s/epoch - 3ms/step
Epoch 2/32
44898/44898 - 118s - loss: 0.1549 - accuracy: 1.0000 - val_loss: 0.1218 - val_accuracy: 1.0000 - 118s/epoch - 3ms/step
Epoch 3/32


KeyboardInterrupt: ignored